This file contains python code to analysis the resutls for comparing with sinica paper

Here I only extract the results simply and put useful result in a single file. 

The raw results take too long to load (on Sep 6, 2023)

In [1]:
import sys
sys.path.append("../mypkg")

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from numbers import Number

from easydict import EasyDict as edict
from tqdm import trange, tqdm
import itertools
from scipy.stats import chi2

from joblib import Parallel, delayed


In [3]:
# This will reload all imports as soon as the code changes
%load_ext autoreload
%autoreload 2

In [4]:
from constants import DATA_ROOT, RES_ROOT, FIG_ROOT, MIDRES_ROOT
from utils.misc import load_pkl, save_pkl

In [5]:
plt.style.use(FIG_ROOT/"base.mplstyle")
torch.set_default_tensor_type(torch.DoubleTensor)

# Param and fns

## Params

In [9]:
paras = edict()
paras.res_dir = RES_ROOT/"simu_linear_sinica_samebetaX_lastbeta02_imp_cvcenter_init0"

In [10]:

def _fil2simres(cur_fil, save_keys=None):
    """ Extract simple result to save to ease the analysis step
    """
    if save_keys is None:
        save_keys = ["cv_errs", "T_vs", "pvals"]
    sim_res = _fname2dict(cur_fil)
    full_res = load_pkl(cur_fil, verbose=False);
    for key in save_keys: 
        sim_res[key] = full_res[key]
    return sim_res

In [11]:
def _fname2dict(fname):
    """From file name to get (seed, lam, N)
    """
    para = list(map(lambda x: float(x.split("_")[-1]), fname.stem.split("-")))
    para1 = edict()
    para1.seed = para[0]
    para1.lam = para[1]/1000
    para1.N = para[2]
    para1.cs = para[3]/10
    return para1

# pre-processing

I save the reults for each N. 

Currently, the simple results include dict_keys(['seed', 'lam', 'N', 'cs', 'cv_errs', 'T_vs', 'pvals'])

Each saved file is a dict including 
    
    - paras: the paras used
    
    - results: a list, each item is also a list for each lam (I keep the order for both layer of lists in a increasing order. 


In [12]:
cur_N = 8
c1v = 0.4
for c1v in [0, 0.1, 0.2, 0.4]:
    for cur_N in [4, 6, 8, 10, 12, 14, 16]:
        
        cur_fils = list(paras.res_dir.glob(f"seed_*-lam_*-N_{cur_N:.0f}-c1_{c1v*1000:.0f}.pkl"));
        res_paras = load_pkl(cur_fils[0], verbose=True).paras;
        all_seeds = sorted(list(set(map(lambda x: int(_fname2dict(x).seed), cur_fils))))
        
        saved_res = edict()
        
        all_res = []
        def _run_fn(cur_seed):
            fils = list(paras.res_dir.glob(f"seed_{cur_seed:.0f}-lam_*-N_{cur_N:.0f}-c1_{c1v*1000:.0f}.pkl"));
            if len(fils) < len(res_paras.can_lams):
                return None
            # sort via lam value
            sorted_fils = sorted(fils, key=lambda x: _fname2dict(x).lam);
            lam_itm = []
            for fil in sorted_fils: 
                sim_res = _fil2simres(fil)
                lam_itm.append(sim_res)
            return lam_itm
        
        #all_seeds = all_seeds[-10:]
        with Parallel(n_jobs=20) as parallel:
            all_res = parallel(delayed(_run_fn)(seed) for seed in tqdm(all_seeds, total=len(all_seeds), desc=f"cv: {c1v:.2f}; N: {cur_N}."))
        saved_res.paras = res_paras
        saved_res.all_res = list(filter(lambda x: x is not None, all_res))
        
        saved_fname = f"allseeds_{len(saved_res.all_res)}-N_{cur_N:.0f}-c1_{c1v*1000:.0f}.pkl"
        
        save_pkl(paras.res_dir/saved_fname, saved_res, is_force=True)

Load file /data/rajlab1/user_data/jin/MyResearch/HDF_infer/notebooks/../mypkg/../results/simu_linear_sinica_samebetaX_lastbeta02_imp_cvcenter_init0/seed_0-lam_300-N_4-c1_0.pkl


cv: 0.00; N: 4.: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:16<00:00, 12.38it/s]


Save to /data/rajlab1/user_data/jin/MyResearch/HDF_infer/notebooks/../mypkg/../results/simu_linear_sinica_samebetaX_lastbeta02_imp_cvcenter_init0/allseeds_200-N_4-c1_0.pkl
Load file /data/rajlab1/user_data/jin/MyResearch/HDF_infer/notebooks/../mypkg/../results/simu_linear_sinica_samebetaX_lastbeta02_imp_cvcenter_init0/seed_0-lam_10-N_6-c1_0.pkl


cv: 0.00; N: 6.: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:33<00:00,  6.03it/s]


Save to /data/rajlab1/user_data/jin/MyResearch/HDF_infer/notebooks/../mypkg/../results/simu_linear_sinica_samebetaX_lastbeta02_imp_cvcenter_init0/allseeds_200-N_6-c1_0.pkl
Load file /data/rajlab1/user_data/jin/MyResearch/HDF_infer/notebooks/../mypkg/../results/simu_linear_sinica_samebetaX_lastbeta02_imp_cvcenter_init0/seed_0-lam_10-N_8-c1_0.pkl


...

Save to /data/rajlab1/user_data/jin/MyResearch/HDF_infer/notebooks/../mypkg/../results/simu_linear_sinica_samebetaX_lastbeta02_imp_cvcenter_init0/allseeds_200-N_12-c1_400.pkl
Load file /data/rajlab1/user_data/jin/MyResearch/HDF_infer/notebooks/../mypkg/../results/simu_linear_sinica_samebetaX_lastbeta02_imp_cvcenter_init0/seed_0-lam_10-N_14-c1_400.pkl


cv: 0.40; N: 14.: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:10<00:00, 18.76it/s]


Save to /data/rajlab1/user_data/jin/MyResearch/HDF_infer/notebooks/../mypkg/../results/simu_linear_sinica_samebetaX_lastbeta02_imp_cvcenter_init0/allseeds_200-N_14-c1_400.pkl
Load file /data/rajlab1/user_data/jin/MyResearch/HDF_infer/notebooks/../mypkg/../results/simu_linear_sinica_samebetaX_lastbeta02_imp_cvcenter_init0/seed_0-lam_10-N_16-c1_400.pkl


cv: 0.40; N: 16.: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:11<00:00, 17.63it/s]


Save to /data/rajlab1/user_data/jin/MyResearch/HDF_infer/notebooks/../mypkg/../results/simu_linear_sinica_samebetaX_lastbeta02_imp_cvcenter_init0/allseeds_200-N_16-c1_400.pkl
